<a href="https://colab.research.google.com/github/sivaratrisrinivas/ttt-playground/blob/main/notebooks/all_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/sivaratrisrinivas/ttt-playground/blob/main/notebooks/all_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TTT Playground - All Tests

Combined notebook for all phase tests. Sections:
1. **Setup** - Clone, install, verify GPU
2. **Phase 2** - Document Processing (PDF, Chunker, Validator)
3. **Phase 3** - TTT-Linear Layer

---
# 1. Setup

In [ ]:
# Clone repo (or pull latest if exists)
import os
if os.path.exists('/content/ttt-playground'):
    !cd /content/ttt-playground && git pull
    %cd /content/ttt-playground
else:
    !git clone https://github.com/sivaratrisrinivas/ttt-playground.git
    %cd ttt-playground

# IMPORTANT: if this runtime previously imported src.*, force reload after git pull
import importlib
import sys
importlib.invalidate_caches()
for _m in [m for m in list(sys.modules.keys()) if m == 'src' or m.startswith('src.')]:
    del sys.modules[_m]
print('✓ Cleared cached src.* modules')

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd()))
print(f"✓ Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt
print("✓ Dependencies installed")

In [ ]:
# Verify GPU
!nvidia-smi
import torch
print(f"\nCUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Verify all imports
import torch
import transformers
import fitz  # PyMuPDF
import gradio
import tiktoken
import tqdm
from loguru import logger
import pydantic
print("✓ All imports successful!")

---
# 2. Phase 2: Document Processing

In [ ]:
# Import document processing modules
from src.document.pdf_parser import PDFParser, PDFExtractionError
from src.document.chunker import DocumentChunker
from src.document.validator import DocumentValidator
from src.config import DocumentConstraints, DocumentChunk
from transformers import AutoTokenizer
import fitz
print("✓ Document processing imports successful")

## 2.1 Generate Test PDFs

In [ ]:
def create_test_pdf(filename: str, num_pages: int, text_per_page: str):
    """Create a test PDF with specified pages and text"""
    doc = fitz.open()
    for i in range(num_pages):
        page = doc.new_page()
        page.insert_text((50, 50), f"Page {i+1}")
        page.insert_text((50, 100), text_per_page)
    doc.save(filename)
    doc.close()
    print(f"Created {filename} ({num_pages} pages)")

text_short = "This is a short test document. " * 50
create_test_pdf("test_short.pdf", 3, text_short)

text_medium = "This is a medium test document with more content. " * 100
create_test_pdf("test_medium.pdf", 20, text_medium)

with open("test_corrupt.pdf", "wb") as f:
    f.write(b"not a valid pdf file")

print("\n✓ Test PDFs created")

## 2.2-2.3 PDFParser Tests

In [ ]:
parser = PDFParser()

# Test valid PDF
with open("test_short.pdf", "rb") as f:
    pdf_bytes = f.read()

text, page_count = parser.parse(pdf_bytes)
print(f"✓ Parsed test_short.pdf:")
print(f"  - Pages: {page_count}")
print(f"  - Text length: {len(text)} chars")
assert page_count > 0 and len(text) > 0

# Test error handling
try:
    with open("test_corrupt.pdf", "rb") as f:
        parser.parse(f.read())
    assert False, "Should have raised PDFExtractionError"
except PDFExtractionError:
    print("✓ Error handling works")

## 2.4-2.6 DocumentChunker Tests

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
chunker = DocumentChunker(tokenizer, chunk_size=2048)
print(f"✓ Chunker initialized with chunk_size={chunker.chunk_size}")

# Test short text (single chunk)
short_text = "This is a short text. " * 10
chunks_short = chunker.chunk(short_text)
print(f"✓ Short text: {len(chunks_short)} chunk(s)")

# Test large text (multiple chunks)
large_text = "word " * 5000
chunks_large = chunker.chunk(large_text)
print(f"✓ Large text (~5000 tokens): {len(chunks_large)} chunks")
for i, chunk in enumerate(chunks_large):
    assert chunk.token_count <= 2048, f"Chunk {i} exceeds limit"

# Verify token preservation
original_ids = tokenizer.encode(large_text, add_special_tokens=False)
reconstructed_ids = []
for chunk in chunks_large:
    reconstructed_ids.extend(chunk.token_ids)
assert reconstructed_ids == original_ids, "Token preservation failed!"
print(f"✓ Token preservation verified: {len(original_ids)} tokens")

## 2.7 DocumentValidator Tests

In [ ]:
validator = DocumentValidator()

with open("test_short.pdf", "rb") as f:
    pdf_bytes = f.read()

# Test valid (relaxed constraints)
is_valid, _ = validator.validate(pdf_bytes, DocumentConstraints(min_tokens=50))
assert is_valid, "Should pass relaxed validation"
print("✓ Valid PDF passes")

# Test max_pages violation
is_valid, msg = validator.validate(pdf_bytes, DocumentConstraints(max_pages=2, min_tokens=50))
assert not is_valid
print(f"✓ max_pages violation detected: {msg}")

# Test min_tokens violation
is_valid, msg = validator.validate(pdf_bytes, DocumentConstraints(min_tokens=500))
assert not is_valid
print(f"✓ min_tokens violation detected: {msg}")

# Test corrupt PDF
with open("test_corrupt.pdf", "rb") as f:
    is_valid, msg = validator.validate(f.read(), DocumentConstraints(min_tokens=50))
assert not is_valid
print(f"✓ Corrupt PDF rejected: {msg}")

print("\n" + "="*50)
print("✓ ALL PHASE 2 TESTS PASSED!")
print("="*50)

---
# 3. Phase 3: TTT-Linear Layer

## 3.1 Import models package

In [ ]:
from src.models import *
print("✓ Step 3.1: from src.models import * succeeds")

## 3.2 TTTLinear.__init__

In [ ]:
from src.models.ttt_linear import TTTLinear
import importlib
import src.models.ttt_linear as _ttt_linear
importlib.reload(_ttt_linear)

layer = TTTLinear(768, 2048, 768)
print(f"✓ TTTLinear instantiated")
print(f"  W_h.shape: {layer.W_h.shape}")
assert layer.W_h.shape == (2048, 768)
print("✓ Step 3.2: W_h.shape == (2048, 768) verified")

## 3.3 TTTLinear.forward (inference mode)

In [ ]:
import torch

x = torch.randn(1, 128, 768)
y = layer(x, learning=False)
print(f"  Input shape: {x.shape}")
print(f"  Output shape: {y.shape}")
assert y.shape == (1, 128, 768)
print("✓ Step 3.3: Output shape [1, 128, 768] verified")

## 3.4 Initial weights stored for reset

In [ ]:
assert hasattr(layer, '_W_h_initial'), "Missing _W_h_initial attribute"
assert torch.allclose(layer.W_h, layer._W_h_initial)
print("✓ Step 3.4: _W_h_initial stored and matches W_h")

## 3.5 TTTLinear.forward (learning mode)

In [ ]:
layer = TTTLinear(768, 2048, 768)
w_before = layer.W_h.clone()

x = torch.randn(1, 128, 768)
y = layer(x, learning=True)

assert not torch.allclose(layer.W_h, w_before), "W_h should change after learning=True"
print("✓ Step 3.5: W_h differs from initial after learning=True")

## 3.6 reset_weights()

In [ ]:
layer.reset_weights()
assert torch.allclose(layer.W_h, layer._W_h_initial)
print("✓ Step 3.6: reset_weights() restores initial W_h")

## 3.7 get_weight_delta()

In [ ]:
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768)
layer(x, learning=True)

delta = layer.get_weight_delta()
print(f"  Weight delta: {delta}")
assert delta > 0
print("✓ Step 3.7: get_weight_delta() > 0 after learning")

## 3.8 Gradient flow

In [ ]:
layer = TTTLinear(768, 2048, 768)
x = torch.randn(1, 128, 768, requires_grad=True)
y = layer(x, learning=False)
loss = y.sum()
loss.backward()

assert x.grad is not None
print("✓ Step 3.8: Gradient flows through layer")

print("\n" + "="*50)
print("✓ ALL PHASE 3 TESTS PASSED!")
print("="*50)

---
# 4. Phase 4: TinyLlama Integration

## 4.1 TTTModel class skeleton

In [ ]:
from src.models.ttt_model import TTTModel
print('✓ Step 4.1: TTTModel class imports successfully')

## 4.2 TTTModel.from_pretrained() - load TinyLlama

In [ ]:
# Load TinyLlama with TTT layers
model = TTTModel.from_pretrained(
    model_name='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device='cuda'
)
print('✓ Step 4.2: TTTModel loaded')

# Test generate
output = model.generate('Hello', max_new_tokens=20)
print(f'  Generated: {output[:100]}...')

## 4.3 Identify MLP layers in TinyLlama

In [ ]:
# Print MLP layer names
print('MLP layers in TinyLlama:')
for name, module in model.model.named_modules():
    if 'mlp' in name.lower():
        print(f'  {name}: {type(module).__name__}')
print('✓ Step 4.3: MLP layers identified')

## 4.4-4.5 Replace MLP with TTTLinear

In [ ]:
# Check that TTT layers were installed
print(f'Number of TTT layers: {len(model.ttt_layers)}')
assert len(model.ttt_layers) > 0, 'Should have TTT layers'
print('✓ Step 4.4-4.5: TTT layers installed')

# Test forward pass still works
output2 = model.generate('The capital of France is', max_new_tokens=10)
print(f'  Generated: {output2}')

## 4.6 All MLP layers replaced

In [ ]:
# TinyLlama has 22 transformer layers
num_layers = len(model.ttt_layers)
print(f'TTT layers: {num_layers}')
# Note: May be fewer if we only replace subset
print('✓ Step 4.6: TTT layer count verified')

## 4.7 reset_learning()

In [ ]:
# Get initial delta (should be 0)
delta_before = model.get_total_weight_delta()
print(f'  Delta before learning: {delta_before}')

# Simulate learning by calling forward with learning=True
# (This would normally be done via learn_from_chunks)
model.reset_learning()
delta_after_reset = model.get_total_weight_delta()
print(f'  Delta after reset: {delta_after_reset}')
assert delta_after_reset == 0, 'Delta should be 0 after reset'
print('✓ Step 4.7: reset_learning() works')

## 4.8 clear_context()

In [ ]:
# Generate, clear, generate again
out1 = model.generate('Test', max_new_tokens=5)
model.clear_context()
out2 = model.generate('Test', max_new_tokens=5)
print(f'  Before clear: {out1}')
print(f'  After clear: {out2}')
print('✓ Step 4.8: clear_context() works')

print('\n' + '='*50)
print('✓ ALL PHASE 4 TESTS PASSED!')
print('='*50)